task name: NormalizeSupervisedBinning
task description: Implement the NormalizeSupervisedBinning example using the provided C# code.

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using static Microsoft.ML.Transforms.NormalizingTransformer;

In [ ]:
var mlContext = new MLContext();

In [ ]:
public class DataPoint
{
    [VectorType(4)]
    public float[] Features { get; set; }

    public string Bin { get; set; }
}

var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[4] { 8, 1, 3, 0}, Bin ="Bin1" },
    new DataPoint(){ Features = new float[4] { 6, 2, 2, 1}, Bin ="Bin2" },
    new DataPoint(){ Features = new float[4] { 5, 3, 0, 2}, Bin ="Bin2" },
    new DataPoint(){ Features = new float[4] { 4,-8, 1, 3}, Bin ="Bin3" },
    new DataPoint(){ Features = new float[4] { 2,-5,-1, 4}, Bin ="Bin3" }
};
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
data = mlContext.Transforms.Conversion.MapValueToKey("Bin").Fit(data).Transform(data);

In [ ]:
var normalize = mlContext.Transforms.NormalizeSupervisedBinning(
    "Features", labelColumnName: "Bin", mininimumExamplesPerBin: 1,
    fixZero: false);
var normalizeFixZero = mlContext.Transforms.NormalizeSupervisedBinning(
    "Features", labelColumnName: "Bin", mininimumExamplesPerBin: 1,
    fixZero: true);

In [ ]:
var normalizeTransform = normalize.Fit(data);
var transformedData = normalizeTransform.Transform(data);
var normalizeFixZeroTransform = normalizeFixZero.Fit(data);
var fixZeroData = normalizeFixZeroTransform.Transform(data);

In [ ]:
var column = transformedData.GetColumn<float[]>("Features").ToArray();
foreach (var row in column)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));
var columnFixZero = fixZeroData.GetColumn<float[]>("Features").ToArray();
foreach (var row in columnFixZero)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));

In [ ]:
var transformParams = normalizeTransform.GetNormalizerModelParameters(0)
    as BinNormalizerModelParameters<ImmutableArray<float>>;
var fixZeroParams = normalizeFixZeroTransform.GetNormalizerModelParameters(0) 
    as BinNormalizerModelParameters<ImmutableArray<float>>;

Console.WriteLine($"The 1-index value in resulting array would be produce by:");
Console.WriteLine("y = (Index(x) / " + transformParams.Density[0] + ") - " + 
    (transformParams.Offset.Length == 0 ? 0 : transformParams.Offset[0]));
Console.WriteLine("Where Index(x) is the index of the bin to which x belongs");
Console.WriteLine("Bins upper borders are: " + string.Join(" ", transformParams.UpperBounds[0]));

Console.WriteLine($"The 1-index value in resulting array would be produce by:");
Console.WriteLine(" y = (Index(x) / " + fixZeroParams.Density[1] + ") - " + 
    (fixZeroParams.Offset.Length == 0 ? 0 : fixZeroParams.Offset[1]));
Console.WriteLine("Where Index(x) is the index of the bin to which x belongs");
Console.WriteLine("Bins upper borders are: " + string.Join(" ", fixZeroParams.UpperBounds[1]));